In [ ]:
import networkx as nx
from tufast_matching_tool.matching import read_sheet
from tufast_matching_tool.general import DAYS,SHIFTS,SHIFT_TYPES,get_shift_index
from tufast_matching_tool.mincost import MatchingGraph

In [ ]:
df, shift_type_dict = read_sheet("../Shiftsplan_lux025_2023-11-06.xlsx")

In [ ]:
matching_graph = MatchingGraph(df,shift_type_dict)

In [ ]:
graph_copy = matching_graph.copy()

#graph_copy.disable_shift(7)
#graph_copy.disable_shift(19)

flow = nx.min_cost_flow(graph_copy.graph,capacity="weight",weight="cost")
graph_copy.get_flow_value()

In [ ]:
flow = matching_graph.get_flow()
retval = None
if flow is None:
    avai_shifts = [get_shift_index(s) for s in shift_type_dict if len(shift_type_dict[s]) > 0]
    for i in range(3):
        retval = matching_graph.get_best_flow_by_remove_shift(avai_shifts,i)
        if retval is not None:
            break

if retval is not None:
    graph_copy = matching_graph.copy()
    for shift in retval[0]:
        graph_copy.disable_shift(get_shift_index(shift))
    flow = graph_copy.get_flow()
else:
    graph_copy = matching_graph.copy()

shift_list = graph_copy.get_shift_list(flow)
shift_list

In [ ]:
# sanity check
for s in shift_list:
    if shift_list[s]["lead"] is not None:
        assert df[df["name"] == shift_list[s]["lead"]][s].values[0] == 1
        is_trained = False
        for t in SHIFT_TYPES:
            is_trained |= df[df["name"] == shift_list[s]["lead"]]["sl_"+t]
        assert is_trained.values[0]
    for w in shift_list[s]["worker"]:
        assert df[df["name"] == w][s].values[0] == 1